In [ ]:
from ruamel.yaml import YAML
import os
import re
import glob

In [ ]:
# Directory containing YAML files
directory_path = 'models'
all_yml_files = list(glob.iglob(os.path.join(directory_path, '**/*.yml'), recursive=True))


In [ ]:
# identifies yamls with invalid syntax
for file_path in all_yml_files:
    try:
        with open(file_path, 'r') as file:
            data = YAML().load(file)
    except Exception as e:
            print(f"error with '{file_path}': {e}")

In [ ]:
def delete_jinja_whitespace_in_ymls(file_path):
    pattern = r'{{\s*(doc\([\'"][^\'"]+[\'"]\s*\))\s*}}'

    try:
        with open(file_path, 'r') as f:
            lines = f.readlines()

        modified_lines = []
        for line in lines:
            modified_lines.append(re.sub(pattern, r'{{\1}}', line))

        with open(file_path, 'w') as f:
            f.writelines(modified_lines)

        print(f"Whitespace in jinja macros removed successfully in '{file_path}'")
    except Exception as e:
        print(f"An error occurred while removing jinja containing whitespace from '{file_path}': {e}")


def create_yaml_file(data, old_file_path,new_file_name):
    """
    Creates a YAML file and write the provided data to it.

    Parameters:
    data (dict): Data to be written to the YAML file.
    old_file_path (str): Name of the YAML file to read from.
    new_file_name (str): Name of the YAML file to be written to, without path.

    Returns:
    bool: True if the file is created and data is written successfully, False otherwise.
    """
    try:
        yaml = YAML()
        new_file_path = os.path.join(os.path.dirname(old_file_path),new_file_name)
        with open(new_file_path, 'w') as new_file:

            # hardcoded yaml version plus some newlines
            new_file.write("version: 2\n\n")

            # use the same indentation as used by dbt in their documentation and in the codegen dbt package
            yaml.indent(mapping=2, sequence=4, offset=2)

            # write the yaml content to a new file
            yaml.dump(data, new_file)

        print(f"File successfully created: '{new_file_path}'")
        return True
    except Exception as e:
        print(f"An error occurred while creating the file '{new_file_path}': {e}")
        return False

def delete_file(file_path):
    """
    Delete a file specified by the file_path.

    Parameters:
    file_path (str): Path to the file.

    Returns:
    bool: True if the file is deleted successfully, False otherwise.
    """
    try:
        os.remove(file_path)
        print(f"File successfully deleted: '{file_path}'")
        return True
    except FileNotFoundError:
        print(f"File not found: '{file_path}'")
        return False
    except Exception as e:
        print(f"An error occurred while deleting the file: {e}")
        return False

In [ ]:
# Iterate through all YAML files recursively in the directory

for file_path in all_yml_files:
    delete_jinja_whitespace_in_ymls(file_path)

for file_path in all_yml_files:
    with open(file_path, 'r') as file:
        data = YAML().load(file)


    # Loop through the "models" section
    for model in data['models']:
        new_data = {'models': [model]}
        filename = f"{model['name']}.yml"

        # Create the new file in the same directory as the original file
        create_yaml_file(new_data, file_path, filename)
    # Delete old files if a new file has been created
    if os.path.basename(file_path) != filename:
        delete_file(file_path)